# 01_MA_AR — UPDATED (2025-10-23 07:40)
Denne notatboken er en **oppdatert** versjon av filen du lastet opp. 
Jeg har **markert alle konkrete endringer** med: `# >>> CHANGE:`

Endringene sikrer at:
- AR og MA er **separate, enkle modeller** (ingen ARMA/ARIMA).
- **AR**: stasjonaritet håndheves og **konstant** (`trend='c'`) er inkludert.
- **MA**: invertibilitet håndheves, konsistent evaluering i **nivå** (med bias-korreksjon).
- Unngår å blande **logg** og **nivå** i evaluering/plot.
- AIC-grid bruker riktige constraints.


In [ ]:
# --- Imports
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller


In [ ]:
# --- Helper functions
def residual_diagnostics(resid, lags=24):
    """Basic diagnostics for ε_t: mean, std, Ljung–Box p-value at given lag."""
    r = pd.Series(resid).dropna()
    lb = acorr_ljungbox(r, lags=[lags], return_df=True)
    return {
        "eps_mean": float(r.mean()),
        "eps_std": float(r.std(ddof=1)),
        f"LjungBox_p(lag{lags})": float(lb["lb_pvalue"].iloc[-1]),
    }

def mae(y, yhat):
    y, yhat = np.asarray(y), np.asarray(yhat)
    return float(np.mean(np.abs(y - yhat)))

def mape(y, yhat):
    y, yhat = np.asarray(y), np.asarray(yhat)
    denom = np.where(y == 0, np.nan, y)
    return float(np.nanmean(np.abs((y - yhat) / denom)) * 100.0)

def adf_p(x):
    x = pd.Series(x).dropna()
    return adfuller(x, autolag="AIC")[1]


In [ ]:
# --- Data: train/test split already prepared as CSVs
train = pd.read_csv("train_data.csv", parse_dates=["date"]).set_index("date").sort_index()
test  = pd.read_csv("test_data.csv",  parse_dates=["date"]).set_index("date").sort_index()

print(f"Train: {train.index.min().date()} → {train.index.max().date()}  rows={len(train)}")
print(f"Test:  {test.index.min().date()}  → {test.index.max().date()}   rows={len(test)}")

# Work on log-volume series (variance stabilization)
train['log_volume'] = np.log(train['volume_all'])
test['log_volume']  = np.log(test['volume_all'])

V_tr = train['log_volume']
V_te = test['log_volume']


In [ ]:
# --- Stationarity diagnostics on full (merged) data
df_full = pd.read_csv("merged_volume_volatility.csv", parse_dates=["date"]).set_index("date").sort_index()
df_full['log_volume'] = np.log(df_full['volume_all'])

p_raw = adfuller(df_full['volume_all'].dropna(), autolag="AIC")[1]
p_log = adfuller(df_full['log_volume'].dropna(), autolag="AIC")[1]
print(f"ADF p-value (raw volume): {p_raw:.5f}")
print(f"ADF p-value (log volume): {p_log:.5f}")

fig, ax = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
df_full['volume_all'].plot(ax=ax[0])
ax[0].set_title("Raw Volume (V_t)")
ax[0].set_ylabel("Volume")
df_full['log_volume'].plot(ax=ax[1])
ax[1].set_title("Log-Transformed Volume (ln(V_t))")
ax[1].set_ylabel("Log Volume")
plt.tight_layout(); plt.show()


## MA(q) — enkel MA på logg-serien (med riktig evaluering i nivå)

In [ ]:
# --- MA(q) model on log-volume
q = 1  # order of the MA(q)

ma_model = sm.tsa.SARIMAX(
    V_tr,
    order=(0, 0, q),        # MA(q)
    trend='c',              # include constant
    enforce_stationarity=False,
    enforce_invertibility=True  # >>> CHANGE: enforce invertibility for MA
)
ma_res = ma_model.fit(disp=False)
print(ma_res.summary().tables[1])

# Forecast for the full test horizon (LOG scale)
fc_ma = ma_res.get_forecast(steps=len(V_te))
sf_ma = fc_ma.summary_frame()
mu_ma = sf_ma['mean']
sigma2_ma = sf_ma['mean_se']**2

# Back-transform to LEVEL with bias correction
# >>> CHANGE: evaluate in LEVEL scale consistently
ma_level = np.exp(mu_ma + 0.5 * sigma2_ma)
y_te_level = np.exp(V_te)

# Diagnostics + metrics
diag_ma = residual_diagnostics(ma_res.resid, lags=24)
print("Diagnostics MA(q):", diag_ma)
print(f"MA(q={ma_res.model_orders['ma']})  MAE(level): {mae(y_te_level, ma_level):.3f}")

# Plot (LEVEL scale)
plt.figure(figsize=(10,5))
plt.plot(V_te.index, y_te_level.values, label="Actual (level)")
plt.plot(V_te.index, ma_level.values, label=f"MA({q}) forecast (level)", linestyle="--")
plt.title("MA Forecast vs Actual (level)")
plt.ylabel("Volume"); plt.legend(); plt.show()


## AR(p) — enkel AR på logg-serien (med konstant & stasjonaritetskrav)

In [ ]:
# --- AR(p) model on log-volume
p = 1

ar_model = sm.tsa.SARIMAX(
    V_tr,
    order=(p, 0, 0),            # AR(p)
    trend='c',                  # >>> CHANGE: include constant c
    enforce_stationarity=True,  # >>> CHANGE: enforce stationarity for AR
    enforce_invertibility=False # (irrelevant for pure AR)
)
ar_res = ar_model.fit(disp=False)
print(ar_res.summary().tables[1])

# Forecast (LOG) and back-transform to LEVEL with bias correction
fc_ar = ar_res.get_forecast(steps=len(V_te))
sf_ar = fc_ar.summary_frame()
mu_ar = sf_ar['mean']
sigma2_ar = sf_ar['mean_se']**2
ar_level = np.exp(mu_ar + 0.5 * sigma2_ar)  # >>> CHANGE: consistent LEVEL evaluation

# Diagnostics + metrics
diag_ar = residual_diagnostics(ar_res.resid, lags=24)
print("Diagnostics AR(p):", diag_ar)
print(f"AR(p={ar_res.model_orders['ar']})  MAE(level): {mae(y_te_level, ar_level):.3f}")

# Plot both (LEVEL)
plt.figure(figsize=(11,5))
plt.plot(V_te.index, y_te_level.values, label="Actual (level)")
plt.plot(V_te.index, ma_level.values, label=f"MA({q}) forecast (level)", linestyle="--")
plt.plot(V_te.index, ar_level.values, label=f"AR({p}) forecast (level)", linestyle=":")
plt.title("Volume forecasts: MA vs AR (level)")
plt.ylabel("Volume"); plt.legend(); plt.tight_layout(); plt.show()


## Modellvalg med AIC — korrigerte constraints for AR og MA

In [ ]:
def aic_grid_ma(y, q_max=4):
    rows = []
    for q in range(1, q_max + 1):
        try:
            r = sm.tsa.SARIMAX(
                y, order=(0, 0, q),
                trend='c',                         # >>> CHANGE: include constant
                enforce_stationarity=False,
                enforce_invertibility=True         # >>> CHANGE: enforce invertibility for MA
            ).fit(disp=False)
            rows.append((q, r.aic))
        except Exception:
            rows.append((q, np.inf))
    return pd.DataFrame(rows, columns=["q", "AIC"]).sort_values("AIC")

def aic_grid_ar(y, p_max=4):
    rows = []
    for p in range(1, p_max + 1):
        try:
            r = sm.tsa.SARIMAX(
                y, order=(p, 0, 0),
                trend='c',                         # >>> CHANGE: include constant
                enforce_stationarity=True,         # >>> CHANGE: enforce stationarity for AR
                enforce_invertibility=False
            ).fit(disp=False)
            rows.append((p, r.aic))
        except Exception:
            rows.append((p, np.inf))
    return pd.DataFrame(rows, columns=["p", "AIC"]).sort_values("AIC")

display(aic_grid_ma(V_tr, q_max=4))
display(aic_grid_ar(V_tr, p_max=4))


## Sammenlikningstabell — evaluering i nivå (konsistent)

In [ ]:
scores = pd.DataFrame({
    "MAE(level)":  [mae(y_te_level, ma_level), mae(y_te_level, ar_level)],
    "MAPE(level)": [mape(y_te_level, ma_level), mape(y_te_level, ar_level)],
}, index=[f"MA(q={ma_res.model_orders['ma']})", f"AR(p={ar_res.model_orders['ar']})"])
scores


## (Valgfritt) Når serien ikke er stasjonær: AR/MA på Δlog
Hvis ADF fortsatt indikerer ikke-stasjonaritet, kan du bruke **enkle AR/MA** på første differanse av logg-serien (vekstrater). 
Dette er *fortsatt separate AR/MA-modeller*, men på en stasjonær transformasjon.


In [ ]:
# >>> OPTIONAL (kommentert ut): bruk Δlog hvis nødvendig
# train['dlog'] = train['log_volume'].diff()
# test['dlog']  = test['log_volume'].diff()
# d_tr = train['dlog'].dropna()
# d_te = test['dlog'].dropna()
# print("ADF p (Δlog):", adfuller(d_tr, autolag="AIC")[1])
#
# # AR på Δlog
# ar_d = sm.tsa.SARIMAX(d_tr, order=(1,0,0), trend='c',
#                       enforce_stationarity=True, enforce_invertibility=False).fit(disp=False)
# # MA på Δlog
# ma_d = sm.tsa.SARIMAX(d_tr, order=(0,0,1), trend='c',
#                       enforce_stationarity=False, enforce_invertibility=True).fit(disp=False)
#
# h = len(d_te)
# fc_ar_d = ar_d.get_forecast(steps=h).predicted_mean
# fc_ma_d = ma_d.get_forecast(steps=h).predicted_mean
#
# last_log = train['log_volume'].iloc[-1]
# ar_log_path = last_log + fc_ar_d.cumsum()
# ma_log_path = last_log + fc_ma_d.cumsum()
# ar_level_d = np.exp(ar_log_path)
# ma_level_d = np.exp(ma_log_path)
#
# y_te_level_d = np.exp(test['log_volume'].iloc[:h])
# print("AR(Δlog) MAE(level):", mae(y_te_level_d, ar_level_d))
# print("MA(Δlog) MAE(level):", mae(y_te_level_d, ma_level_d))
